# About improving on a greedy way to increment toward to long term optimum

Joaquim Gromicho, March 2024

## Recall the Timor-Leste assignment from AABW

In [ ]:
!curl -s -H 'Cache-Control: no-cache, no-store' https://raw.githubusercontent.com/gromicho/data/main/AABW/util_AABW.py > util.py

In [ ]:
import util

In [ ]:
data_file = 'DataErmeraTimorLeste.xlsx'
assert util.RetrieveDataSet(data_file)
data = util.ReadWorkbookIntoNamedTuple(data_file,idx_col=0)

distances = data.Distances
homes     = data.Homes
locations = data.PotentialLocations

# Ensure preconditions

In [ ]:
import subprocess, sys, shutil

In [ ]:
def GetListOfInstalledPackages():
    res = subprocess.check_output([sys.executable, '-m', 'pip', 'freeze'])
    return { r.decode().split('==')[0] for r in res.split() }

In [ ]:
def EnsurePreConditionsAtColab():
    at_colab = "google.colab" in sys.modules
    if at_colab:
        installed = GetListOfInstalledPackages()
        if not 'pyomo' in installed:
            %pip install -q pyomo
        if not 'highspy' in installed:
            %pip install -q highspy
        if not shutil.which('/usr/bin/cbc'):
            !apt-get install -y -qq coinor-cbc
            assert(shutil.which('/usr/bin/cbc'))
    return at_colab

In [ ]:
at_colab = EnsurePreConditionsAtColab()

In [ ]:
from pathlib import Path
import itertools as it
from time import perf_counter as pc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyomo.environ as pyo

In [ ]:
def ListAvailableSolvers():
    shell_command = "pyomo help --solvers"
    output = subprocess.check_output(shell_command, shell=True).decode()
    return [
        line.strip()[1:]
        for line in output.split()
        if line.strip().startswith("+") and not line.strip().endswith(")")
    ]

In [ ]:
available_solvers = ListAvailableSolvers()

In [ ]:
distances.shape

# Visualize the data

In [ ]:
def ShowFacilityLocation( xC, yC, xF, yF, X=[], Y=[], file_name=None ):
    plt.plot( xC,yC, '.', color='lightgray' )
    plt.plot( xF,yF, 's', mfc='none', color='cyan' )
    plt.plot( xC[Y],yC[Y], 'o', color='g' )
    plt.plot( xF[X],yF[X], 's', color='y' )
    plt.xticks([])
    plt.yticks([])
    plt.axis('equal')
    if file_name:
        plt.savefig(file_name, bbox_inches='tight')
    plt.show()

In [ ]:
xC = homes['lon']
yC = homes['lat']
xF = locations['lon']
yF = locations['lat']
ShowFacilityLocation( xC, yC, xF, yF, file_name='dataTL.pdf' )

# Prepare the data for optimization

*_In preparing the data prior to building the optimization model we differ from the resolution given in AABW!_*

Let us denote by $J$ the set of possible locations for health facilities and by $I$ the set of household locations: in our Ermera data set $|J| = 94$ and $|I| = 1084$.

The `homes` and the `distances` are indexed in $I$ while the `locations` are indexed in $J$ which are also the columns of `distances`.

After choosing $t$ a threshold on the distance, we define $J_i = \{j \in J : d_{ij} \leq t\}$ as the set of those $j \in J$ for which $d_{ij} \leq t$.
These are the (potential) facilities within reach of each household.

We store these sets as a `dict` called `JI` with keys $i \in I$ and #J_i$ as values stored as `np.array` of unique `integers`.

We also revert the logic to define `IJ` as being the households within the threshold of each (potential) facility. We can see `IJ` as the _catchment areas_ for that threshold.

In [ ]:
def catchment(distances, threshold):
    return {
        i: distances.columns[np.where(row <= threshold)[0]].values
                    for i, row in distances.iterrows()
    }

In [ ]:
def reverse( mapping ):
    from collections import defaultdict
    result = defaultdict(set)
    for x, Y in mapping.items():
        for y in Y:
            result[y].add(x)
    return { y : np.array(sorted(result[y]),dtype=int) for y in sorted(result.keys()) }

In [ ]:
def PrepareOptimizationData( locations, homes, distances, threshold):

    assert all( homes.index == distances.index )
    assert all( locations.index == distances.columns )

    I = homes.index.values
    J = locations.index.values

    JI = catchment( distances, threshold )
    IJ = reverse(JI)

    assert set(JI.keys()).issubset( set(I) )
    assert set(IJ.keys()).issubset( set(J) )
    assert set(np.concatenate(list(IJ.values()))).issubset( set(I) )
    assert set(np.concatenate(list(JI.values()))).issubset( set(J) )

    return I, J, IJ, JI

In [ ]:
I, J, IJ, JI = PrepareOptimizationData( locations, homes, distances, 3 )
w = np.ones_like(I)

### Model `maximal covering` as in the paper by [Church and ReVelle](http://www.geog.ucsb.edu/~forest/G294download/MAX_COVER_RLC_CSR.pdf)

This model defines variables $z_i$ for each household $i\in I$ to indicate if that household can be served by a hospital that is opened.

\begin{align*}
    \max\quad & \sum_{i\in I} v_iz_i  \\
    \text{subject to}\quad & z_i \leq \sum_{j\in J_i } x_j & \forall i \in I \\
    & \sum_{j \in J} x_j \leq p \\
    & x_j \in \{0,1\} & \forall j \in J \\
    & z_i \in \{0,1\} & \forall i \in I
\end{align*}

In [ ]:
def model_max_covering(w, I, J, JI, p):
    m = pyo.ConcreteModel('ModelMaxCovering')

    m.p = pyo.Param(mutable=True, default=p)
    m.I = pyo.Set(initialize=I)
    m.J = pyo.Set(initialize=J)

    @m.Param(m.I)
    def w(m, i):
        return w[i]

    @m.Param(m.I, within=pyo.Any)
    def JI(m, i):
        return JI[i]

    m.x = pyo.Var(m.J, within=pyo.Binary)
    m.z = pyo.Var(m.I, within=pyo.Binary)

    @m.Objective(sense=pyo.maximize)
    def obj(m):
        return pyo.quicksum(m.w[i] * m.z[i] for i in m.I)

    @m.Constraint()
    def budget(m):
        return pyo.quicksum(m.x[j] for j in m.J) <= m.p

    @m.Constraint(m.I)
    def serve_if_reachable_and_open(m, i):
        return m.z[i] <= pyo.quicksum(m.x[j] for j in m.JI[i])

    return m

In [ ]:
def get_selected(x):
    return [j for j, v in x.items() if v() > 0.5]

In [ ]:
if at_colab:
    solvers = ['cbc','appsi_highs']
else:
    solvers = [
         'gurobi_direct'
        ,'mosek_direct'
        ,'cplex_direct'
        ,'cbc'
        ,'glpk'
        ,'appsi_highs'
    ]

solvers =sorted(set(solvers) & set(available_solvers) )
solvers

In [ ]:
t = pc()
m = model_max_covering( w, I, J, JI, 5 )
t = pc()-t
print(f"{'-'*50}\n{model_max_covering.__name__:>30s} {t:5.2f}")
time_per_solver = dict()
for solver in solvers:
    t = pc()
    r = pyo.SolverFactory(solver).solve(m)
    t = pc()-t
    time_per_solver[solver] = t
    print(f'{solver:>30s} {t:5.2f} {m.obj():6.1f}')

In [ ]:
fastest_solver = min(time_per_solver, key=time_per_solver.get)
fastest_solver

In [ ]:
ShowFacilityLocation( xC, yC, xF, yF, get_selected( m.x ), get_selected( m.z ) )

## Pareto frontiers

One can see the maximization of the covering and the budget allocation as somehow multiple objectives: the highest covering for the lowest budget.

In fact, so far, the budget was given as a fact. But, in reality, the decision makers make tradeoffs.

We will not dwell into multi-objective optimization now, but let us use the generation of a Pareto frontier as an illustration for mutable `pyomo` parameters.

In [ ]:
def ComputePareto( w, I, J, JI, p_max, solver, progress=lambda x : x ):
    result = dict()
    start = pc()
    model = model_max_covering( w, I, J, JI, p_max )
    for model.p in progress(range(p_max)):
        modeling = pc()-start
        start = pc()
        solver_result = solver.solve(model, tee=False)
        solving = pc()-start
        result[model.p()] = dict(
            modeling=modeling,
            solving=solving,
            value=model.obj(),
            solution=get_selected( model.x ),
            termination=solver_result.solver.termination_condition,
            upper=solver_result.problem.upper_bound)
        start = pc()
    return result

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
result_pareto = ComputePareto( w, I, J, JI, p_max = len(locations), solver=pyo.SolverFactory(fastest_solver), progress=tqdm )

In [ ]:
opt_results = pd.DataFrame.from_dict( result_pareto, orient='index' )
total_population = w.sum()
opt_results['coverage'] = opt_results['value'] / total_population * 100
opt_results[['coverage']].plot(style='.-',figsize=(10,3))

# A fast implementation of `Greedy`

Note that the implementation below is very efficient and improves on the implementation described in the [thesis by Fleur](https://www.vvsor.nl/articles/jan-hemelrijk-award-winner-2023/).

In [ ]:
def fast_greedy(w, JI, IJ):
    J = list(IJ.keys())

    gain = np.zeros(max(J)+1)
    coverage = np.zeros_like(w, dtype=np.uint16)

    related = { j : np.unique(np.concatenate([JI[i] for i in IJ[j]]))
                for j in J if len(IJ[j])
    }

    def mutation( w, cov, IJ, J, v ):
        return np.array( [w[IJ[j][cov[IJ[j]]==v]].sum() for j in J] )

    sol = []
    gain[J] = mutation( w, coverage, IJ, J, 0 )
    while True:
        s = gain.argmax()
        if gain[s] <= 0:
            break
        coverage[IJ[s]] += 1
        gain[related[s]] = mutation( w, coverage, IJ, related[s], 0 )
        sol.append(s)
    return np.array(sol)

In [ ]:
greedy_sequence = fast_greedy( w, JI, IJ)

In [ ]:
def add_greedy_columns( results, greedy_sequence, IJ, w, prefix ):
    results[f'{prefix}_value'] = [
            w[np.unique( np.concatenate(
                            [IJ[j] for j in greedy_sequence[:i] ]
                        ) )
              ].sum() if i else 0 for i in results.index
        ]

    results[f'{prefix}_coverage'] = results[f'{prefix}_value'] / w.sum() * 100
    return results

In [ ]:
opt_results = add_greedy_columns( opt_results, greedy_sequence, IJ, w, 'greedy' )

In [ ]:
opt_results[['coverage','greedy_coverage']].plot(style='.-',figsize=(10,3))

In [ ]:
( opt_results.coverage - opt_results.greedy_coverage ).max()

## Is Greedy always optimal?

One example may be misleading!

This is the moment that you think again: is $P = NP$ after all?

Of course not. A simple example will do:

In [ ]:
def FinishInstance( homes, locations ):
    from scipy.spatial.distance import cdist
    distances = pd.DataFrame(cdist(homes,locations)) / 100
    homes = pd.DataFrame(homes,columns=['lat','lon'])
    locations = pd.DataFrame(locations,columns=['lat','lon'])
    xC = homes['lon']
    yC = homes['lat']
    xF = locations['lon']
    yF = locations['lat']
    return distances,homes,locations, xC, yC, xF, yF

In [ ]:
homes = [
     [2,0]
    ,[2,2]
    ,[2,4]
    ,[2,6]
    ,[1,1]
    ,[1,3]
    ,[1,5]
    ,[3,1]
    ,[3,3]
    ,[3,5]
]

locations = [
     [2,1]
    ,[2,3]
    ,[2,5]
]
distances,homes,locations, xC, yC, xF, yF = FinishInstance( homes, locations )
distances *= 100
with plt.rc_context({'figure.figsize': (2, 2)}):
  ShowFacilityLocation( xC, yC, xF, yF )

In [ ]:
I, J, IJ, JI = PrepareOptimizationData( locations, homes, distances, 2.5 )
w = np.ones_like(I)

In [ ]:
result_pareto = ComputePareto( w, I, J, JI, p_max = len(locations), solver=pyo.SolverFactory(fastest_solver), progress=tqdm )

In [ ]:
opt_results = pd.DataFrame.from_dict( result_pareto, orient='index' )
total_population = w.sum()
opt_results['coverage'] = opt_results['value'] / total_population * 100
opt_results[['coverage']].plot(style='.-',figsize=(10,3))

In [ ]:
greedy_sequence = fast_greedy( w, JI, IJ)

In [ ]:
opt_results = add_greedy_columns( opt_results, greedy_sequence, IJ, w, 'greedy' )

In [ ]:
opt_results[['coverage','greedy_coverage']].plot(style='.-',figsize=(10,3))

# The approach mentioned in the Nepal paper

 1. Reduce the instance to the solution that is considered optimal in the long term
 1. Apply Greedy to this smaller instance

In [ ]:
best = opt_results.solution.values[-1]

In [ ]:
def reduce_to( I, J, IJ, JI, best ):
    J = np.array(best)
    IJ = { j : IJ[j] for j in J }
    I = np.unique( np.concatenate( list(IJ.values()) ))
    JI = { i : np.array(sorted(set(J).intersection(set(j)))) for i,j in JI.items() }
    return I, J, IJ, JI

In [ ]:
I, J, IJ, JI = reduce_to( I, J, IJ, JI, best )

In [ ]:
new_greedy_sequence = fast_greedy( w, JI, IJ)

In [ ]:
opt_results = add_greedy_columns( opt_results, new_greedy_sequence, IJ, w, 'new_greedy' )

In [ ]:
opt_results[['coverage','greedy_coverage', 'new_greedy_coverage']].plot(style='.-',figsize=(10,3))

In [ ]:
ax = opt_results[['coverage','new_greedy_coverage']].rename(columns={'coverage':'Pareto','new_greedy_coverage':'Greedy deployment'}).plot(style='.-',figsize=(10,3))
ax.set_xticks(range(3))
plt.show()

The research concerns the area between `new_greedy_coverage` and `coverage`.